### Segmenting and Clustering Neighborhoods in Toronto
In this notebook, I am going to scrap data from wikipedia to get the list of neighborhoods in Toronto and then use Square API to cluster these neighborhoods based on the venues they contain.

## Part 1 : Getting the neighborhoods of Toronto

In [1]:
import pandas as pd

In [2]:
from IPython.display import IFrame
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
IFrame(url, width=800, height=350)

#### Using this wikipedia webpage, we will create a dataframe containing three columns: PostalCode, Borough, and Neighborhood

In [3]:
data, = pd.read_html(url, match="Postal", skiprows=1)
data.columns = ["PostalCode", "Borough", "Neighborhood"]
data.head()

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,NaN
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


Let's drop all the cells that don't have any Borough assigned to them:

In [4]:
data = data[data["Borough"] != "Not assigned"]
data.head()

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
data['PostalCode'].describe()

count     103
unique    103
top       M1J
freq        1
Name: PostalCode, dtype: object

As we can notice, there is 103 different Postal Code. 
Each Postal code is associated with a Borough and a certain number of neighborhoods seperated by a comma.

Fortunately, each Borough has at least a neighborhood associated with it, so we don't need further data cleaning.

In [6]:
data[data["Neighborhood"] == "Not assigned"]

,PostalCode,Borough,Neighborhood


In [7]:
data[data["Neighborhood"] == ""]

,PostalCode,Borough,Neighborhood


In [8]:
data.shape

(103, 3)

## Part 2:

In [9]:
locations = pd.read_csv("https://cocl.us/Geospatial_data")
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
locations.columns = ["PostalCode", "Latitude", "Longitude"]
data = pd.merge(data, locations, on='PostalCode')
data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
data.shape

(103, 5)

In [13]:
subset = data[data['Borough'].str.contains("Toronto")]
subset.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [14]:
subset.shape

(39, 5)